<a href="https://colab.research.google.com/github/hassanfv/hfvSPH_on_Unified_Memory_GPU/blob/main/smoothing_h_on_Unified_Mem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile mtest.cu
#include <iostream> // iostream, fstream, cmath, string, vector, sstream.
#include <fstream>
#include <cmath>
#include <string>
#include <vector>
#include <sstream>
#include "myCppSPHLibs.h"
using namespace std;

const int N = 131504;
const int Nngb = 64;
const int Ndown = Nngb - 5;
const int Nup = Nngb + 5;
const float coeff = 0.001;


int main(){

  // Reading Hydra file.
  string fname = "Hydra_130k.csv";

  vector<vector<string>> content;
  vector<string> row;
  string line, word;
  
  fstream file (fname, ios::in);
  if(file.is_open())
  {
  while(getline(file, line))
  {
  row.clear();
  
  stringstream str(line);
  
  while(getline(str, word, ','))
  row.push_back(word);
  content.push_back(row);
  }
  }
  else
  cout<<"Could not open the file\n";

  // Allocate Unified Memory - accessible from CPU and GPU
  float *x, *y, *z, *hres, *hprevious;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  cudaMallocManaged(&z, N*sizeof(float));

  cudaMallocManaged(&hres, N*sizeof(float));
  cudaMallocManaged(&hprevious, N*sizeof(float));

  // 0  1  2  3   4   5    6   7  8  9  10
  // x, y, z, vx, vy, vz, rho, P, c, h, m.

  // Initialize x, y, and z on the Host.
  for(int i = 0; i < N; i++){
    x[i] = stof(content[i][0]);
    y[i] = stof(content[i][1]);
    z[i] = stof(content[i][2]);
  }

  // Initialize hres and hprevious on the Host
  for(int i = 0; i < N; i++){
    hres[i] = 10.1f; // 100.0 is just a place holder!
    hprevious[i] = stof(content[i][9]);;
  }

  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  smoothing_h<<<gridSize, blockSize>>>(x, y, z, hres, hprevious,
                                       N, Ndown, Nup, coeff);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // visual inspection
  for(int i = N-10; i < N; i++){
    cout << hprevious[i] << "  " << hres[i] << endl;
  }

  // Free memory
  cudaFree(x);
  cudaFree(y);
  cudaFree(z);
  cudaFree(hres);
  cudaFree(hprevious);

}

Writing mtest.cu


In [2]:
%%shell
nvcc mtest.cu -o mtest

In [3]:
%%shell
./mtest

0.0491951  0.0491951
0.0541674  0.0541674
0.0506397  0.0506397
0.0476966  0.0476966
0.0563585  0.0563585
0.0607407  0.0607407
0.0579519  0.0579519
0.0439055  0.0439055
0.0553372  0.0553372
0.0540047  0.0540047


In [4]:
%%shell
nvprof ./mtest

==215== NVPROF is profiling process 215, command: ./mtest
0.0491951  0.0491951
0.0541674  0.0541674
0.0506397  0.0506397
0.0476966  0.0476966
0.0563585  0.0563585
0.0607407  0.0607407
0.0579519  0.0579519
0.0439055  0.0439055
0.0553372  0.0553372
0.0540047  0.0540047
==215== Profiling application: ./mtest
==215== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  179.66ms         1  179.66ms  179.66ms  179.66ms  smoothing_h(float*, float*, float*, float*, float*, int, int, int, float)
      API calls:   66.09%  352.39ms         5  70.477ms  4.7300us  352.30ms  cudaMallocManaged
                   33.70%  179.67ms         1  179.67ms  179.67ms  179.67ms  cudaDeviceSynchronize
                    0.09%  489.73us         5  97.945us  10.507us  238.67us  cudaFree
                    0.08%  401.39us         1  401.39us  401.39us  401.39us  cuDeviceTotalMem
                    0.03%  177.21us       101  1.7540us     